# PROCESADO DEL CSV ORIGINAL DE KAGGLE PARA SACAR EL DATAFRAME TRAIN

In [111]:
import pandas as pd
import numpy as np

## Cargamos los datos de entrada del archivo .csv Train

In [112]:
# Cargamos los datos de entrada del archivo Train.csv
test = pd.read_csv('../data/Raw/test.csv', sep=',')
df_test=pd.DataFrame(test)
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [113]:
df_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [114]:
# Hay 2 opciones en la columna 'HomePlanet'. Como son campos de tipo object, osea, string, los paso a 0 ó 1 con el metodo getdummies y los guardo en un dataframe temporal.
planetas=pd.get_dummies(df_test['HomePlanet'])
planetas

,Earth,Europa,Mars
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
4272,1,0,0
4273,1,0,0
4274,0,0,1
4275,0,1,0


In [115]:
df_test=pd.concat([df_test, planetas], axis=1)

In [116]:
df_test=df_test.fillna(df_test.median())
df_test['Cabin']= df_test['Cabin'].astype('string')

c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [117]:
df_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep         0
Cabin           100
Destination      92
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name             94
Earth             0
Europa            0
Mars              0
dtype: int64

In [118]:
# Separo la columna cabin en 3 para crear mas tarde 3 nuevas columnas con sus correspondientes valores sacados de esta
df_test['Cabin_Side'] = df_test['Cabin'].str.split("/").str[2]
df_test['Cabin_num'] = df_test['Cabin'].str.split("/").str[1]
df_test['Cabin_Deck'] = df_test['Cabin'].str.split("/").str[0]

In [119]:
# Separo la columna 'Cabin_Side' en 2, porq tiene 2 valores,P y S y las guardo 
cabinas = pd.get_dummies(df_test['Cabin_Side'], prefix='Cabin_Side')
df_test=pd.concat([df_test, cabinas], axis=1)

In [120]:
# Convertimos la columna Cabin_deck a numerica con getdummies y la guardo en columnas individuales separadas
cabinas = pd.get_dummies(df_test['Cabin_Deck'], prefix='Cabin_Deck')
df_test=pd.concat([df_test, cabinas], axis=1)

In [121]:
# El campo 'Cabin_num', tiene valores . Lo relleno provisionalmente cn nueves y despues con NAN que son valores nulos pero de typo Float :)
df_test['Cabin_num']=df_test['Cabin_num'].fillna('-99999999')
df_test['Cabin_num'].replace({"-99999999": np.nan})
df_test['Cabin_num'].value_counts

<bound method IndexOpsMixin.value_counts of 0               3
1               4
2               0
3               1
4               5
          ...    
4272         1496
4273    -99999999
4274          296
4275          297
4276         1498
Name: Cabin_num, Length: 4277, dtype: object>

In [122]:
#Borramos las columnas que he ido trasformando y que ya no me sirven
df_test.drop('Cabin_Side', axis=1, inplace=True)
df_test.drop('HomePlanet', axis=1, inplace=True)
df_test.drop('Destination', axis=1, inplace=True)
df_test.drop('Name', axis=1, inplace=True)
df_test.drop('Cabin',axis=1, inplace=True)
df_test.drop('Cabin_Deck',axis=1, inplace=True)

#Transformamos la columna VIP, q esta cm string a tipo int
df_test['VIP'] = df_test['VIP'].replace({False: 0, True: 1}).astype(int)
df_test['Cabin_num']=df_test['Cabin_num'].astype(int)

#Transformamos la columna CryoSleep, q esta cm string a tipo int
df_test['CryoSleep'] = df_test['CryoSleep'].replace({False: 0, True: 1}).astype(int)
df_test['Cabin_num']=df_test['Cabin_num'].astype(int)

df_test=df_test.fillna(df_test.median())
df_test = df_test.dropna()


In [123]:
# Ahora si...ataframe limpio de valores nulos y sin tipos de campos String o Object oncompatibles a la hora de crear el modelo y entrenarlo
df_test.isnull().sum()

PassengerId     0
CryoSleep       0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Earth           0
Europa          0
Mars            0
Cabin_num       0
Cabin_Side_P    0
Cabin_Side_S    0
Cabin_Deck_A    0
Cabin_Deck_B    0
Cabin_Deck_C    0
Cabin_Deck_D    0
Cabin_Deck_E    0
Cabin_Deck_F    0
Cabin_Deck_G    0
Cabin_Deck_T    0
dtype: int64

In [124]:
df_test

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,Cabin_Side_P,Cabin_Side_S,Cabin_Deck_A,Cabin_Deck_B,Cabin_Deck_C,Cabin_Deck_D,Cabin_Deck_E,Cabin_Deck_F,Cabin_Deck_G,Cabin_Deck_T
0,0013_01,1,27.0,0,0.0,0.0,0.0,0.0,0.0,1,...,0,1,0,0,0,0,0,0,1,0
1,0018_01,0,19.0,0,0.0,9.0,0.0,2823.0,0.0,1,...,0,1,0,0,0,0,0,1,0,0
2,0019_01,1,31.0,0,0.0,0.0,0.0,0.0,0.0,0,...,0,1,0,0,1,0,0,0,0,0
3,0021_01,0,38.0,0,0.0,6652.0,0.0,181.0,585.0,0,...,0,1,0,0,1,0,0,0,0,0
4,0023_01,0,20.0,0,10.0,0.0,635.0,0.0,0.0,1,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.0,0,0.0,0.0,0.0,0.0,0.0,1,...,0,1,0,0,0,0,0,0,1,0
4273,9269_01,0,42.0,0,0.0,847.0,17.0,10.0,144.0,1,...,0,0,0,0,0,0,0,0,0,0
4274,9271_01,1,26.0,0,0.0,0.0,0.0,0.0,0.0,0,...,1,0,0,0,0,1,0,0,0,0
4275,9273_01,0,26.0,0,0.0,2680.0,0.0,0.0,523.0,0,...,1,0,0,0,0,1,0,0,0,0


In [125]:
# Ahora si, todas las features que voy a entrenar son de typo numerico o bool y sin nulos
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4277 entries, 0 to 4276
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   CryoSleep     4277 non-null   int32  
 2   Age           4277 non-null   float64
 3   VIP           4277 non-null   int32  
 4   RoomService   4277 non-null   float64
 5   FoodCourt     4277 non-null   float64
 6   ShoppingMall  4277 non-null   float64
 7   Spa           4277 non-null   float64
 8   VRDeck        4277 non-null   float64
 9   Earth         4277 non-null   uint8  
 10  Europa        4277 non-null   uint8  
 11  Mars          4277 non-null   uint8  
 12  Cabin_num     4277 non-null   int32  
 13  Cabin_Side_P  4277 non-null   uint8  
 14  Cabin_Side_S  4277 non-null   uint8  
 15  Cabin_Deck_A  4277 non-null   uint8  
 16  Cabin_Deck_B  4277 non-null   uint8  
 17  Cabin_Deck_C  4277 non-null   uint8  
 18  Cabin_Deck_D  4277 non-null 

In [126]:
# guardamos tod procesado en un CSV nuevo para poder cargar los dataframes en los notebooks de los modelos
df_test.to_csv('../data/Processed/test_processed.csv', sep=',', index=False)